In [1]:
import numpy as np
import pymultinest

import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline

import json

import corner

import retrieval_base.auxiliary_functions as af

def read_results(prefix, n_params):

    # Set-up analyzer object
    analyzer = pymultinest.Analyzer(
        n_params=n_params, 
        outputfiles_basename=prefix
        )
    stats = analyzer.get_stats()

    # Load the equally-weighted posterior distribution
    posterior = analyzer.get_equal_weighted_posterior()
    posterior = posterior[:,:-1]

    # Read the parameters of the best-fitting model
    bestfit = np.array(stats['modes'][0]['maximum a posterior'])

    PT = af.pickle_load(prefix+'data/bestfit_PT.pkl')
    Chem = af.pickle_load(prefix+'data/bestfit_Chem.pkl')

    m_spec = af.pickle_load(prefix+'data/bestfit_m_spec.pkl')
    d_spec = af.pickle_load(prefix+'data/d_spec.pkl')

    LogLike = af.pickle_load(prefix+'data/bestfit_LogLike.pkl')

    try:
        Cov = af.pickle_load(prefix+'data/bestfit_Cov.pkl')
    except:
        Cov = None

    int_contr_em           = np.load(prefix+'data/bestfit_int_contr_em.npy')
    int_contr_em_per_order = np.load(prefix+'data/bestfit_int_contr_em_per_order.npy')
    int_opa_cloud          = np.load(prefix+'data/bestfit_int_opa_cloud.npy')

    f = open(prefix+'data/bestfit.json')
    bestfit_params = json.load(f)
    f.close()

    print(posterior.shape)
    return posterior, bestfit, PT, Chem, int_contr_em, int_contr_em_per_order, int_opa_cloud, m_spec, d_spec, LogLike, Cov, bestfit_params

In [2]:
res = read_results(
    prefix='./retrieval_outputs/DENIS_J0255_nominal_5/test_', n_params=32
    )
posterior_1, bestfit_1, PT_1, Chem_1, int_contr_em_1, int_contr_em_per_order_1, int_opa_cloud_1, m_spec_1, d_spec_1, LogLike_1, Cov_1, bestfit_params_1 = res

  analysing data from ./retrieval_outputs/DENIS_J0255_nominal_5/test_.txt
(4331, 32)


In [21]:
bestfit_color = plt.get_cmap('Oranges')(0.6)

#orders_for_zoom_in = [2,5]
orders_for_zoom_in = [5]

hspace = 0.12
gridspec_kw={
    'hspace': 0.0, 
    'left': 0.08, 'right': 0.96,  
    'bottom': 0.08, 'top': 0.95, 
    'height_ratios': [0.7,0.4*0.35, hspace, 0.4*1,0.4*0.35]
    }
fig, ax = plt.subplots(
    #figsize=(10,10), nrows=3*2+2, 
    figsize=(10,7), nrows=2*2+1,    
    gridspec_kw=gridspec_kw
    )

ax_spec = np.array(ax[0::3])
ax_res = np.array(ax[1::3])
for ax_i in ax[2::3]:
    ax_i.remove()

spine_lw = 1.75
for ax_i in ax_spec:
    ax_i.set(xticks=[])
    ax_i.spines[['left','right','top']].set_linewidth(spine_lw)
    
for ax_i in ax_res:
    ax_i.spines[['left','right','bottom']].set_linewidth(spine_lw)

    ax_i.set_zorder(-1)

ax_spec[0].plot(d_spec_1.wave.flatten(), d_spec_1.flux.flatten(), c='k', lw=0.5)
for i in range(d_spec_1.n_orders):
    for j in range(d_spec_1.n_dets):
        ax_spec[0].plot(d_spec_1.wave[i,j], LogLike_1.f[i,j]*m_spec_1.flux[i,j], c=bestfit_color, lw=1)
        
        ax_res[0].plot(d_spec_1.wave[i,j], (d_spec_1.flux[i,j] - LogLike_1.f[i,j]*m_spec_1.flux[i,j]), c='k', lw=0.5)
        ax_res[0].plot(d_spec_1.wave[i,j], 0*m_spec_1.flux[i,j], c=bestfit_color, lw=2)
    
    errorbar_kwargs = {'elinewidth':1.8, 'capsize':2.8, 'capthick':1.8}
    ax_res[0].errorbar(
        d_spec_1.wave[i].min()-3.5, [0], yerr=np.nanmean(d_spec_1.err[i]), c='k', **errorbar_kwargs
        )
    #yerr = np.nanmean(
    #    np.concatenate([(LogLike_1.beta[i,k]**2 * Cov_1[i,k].cov).diagonal() for k in range(3)])
    #    )**(1/2)
    yerr = np.nanmean(
        np.concatenate([np.sqrt(LogLike_1.beta[i,k]**2 * Cov_1[i,k].cov).diagonal() for k in range(3)])
        )
    print(np.nanmean(d_spec_1.err[i]) / yerr)
    ax_res[0].errorbar(
        d_spec_1.wave[i].min()-9.5, [0], yerr=yerr, c=bestfit_color, **errorbar_kwargs
        )

for i, order in enumerate(orders_for_zoom_in):
    ax_spec[i+1].plot(
        d_spec_1.wave[order,:].flatten(), d_spec_1.flux[order,:].flatten(), c='k', lw=0.5
        )

    for j in range(d_spec_1.n_dets):
        ax_spec[i+1].plot(d_spec_1.wave[order,j], LogLike_1.f[order,j]*m_spec_1.flux[order,j], c=bestfit_color, lw=2)

        ax_res[i+1].plot(d_spec_1.wave[order,j], (d_spec_1.flux[order,j] - LogLike_1.f[order,j]*m_spec_1.flux[order,j]), c='k', lw=0.5)
        ax_res[i+1].plot(d_spec_1.wave[order,j], 0*m_spec_1.flux[order,j], c=bestfit_color, lw=2)
        
    ax_res[i+1].errorbar(
        d_spec_1.wave[order].min()-0.5, [0], yerr=np.nanmean(d_spec_1.err[order]), c='k', **errorbar_kwargs
        )
    #yerr = np.nanmean(
    #    np.concatenate([(LogLike_1.beta[i,k]**2 * Cov_1[order,k].cov).diagonal() for k in range(3)])
    #    )**(1/2)
    yerr = np.nanmean(
        np.concatenate([np.sqrt(LogLike_1.beta[i,k]**2 * Cov_1[order,k].cov).diagonal() for k in range(3)])
        )
    print(yerr)
    ax_res[i+1].errorbar(
        d_spec_1.wave[order].min()-1.05, [0], yerr=yerr, c=bestfit_color, **errorbar_kwargs
        )
    
    x0 = ax_spec[i+1].get_xlim()[0]
    y0 = ax_spec[i+1].get_ylim()[0]
    width  = np.abs(ax_spec[i+1].get_xlim()[1] - x0)
    height = np.abs(ax_spec[i+1].get_ylim()[1] - y0)
    bounds = (x0, y0, width, height)

    _, con_patch = ax_spec[0].indicate_inset(
        bounds=bounds,
        inset_ax=ax_spec[i+1], #inset_ax=None, 
        edgecolor='k', alpha=0.2, 
        )
    #for con_patch_k in con_patch:
    #    con_patch_k.set(zorder=-1, clip_on=False)

    height_ratio = gridspec_kw['height_ratios'][-1] / gridspec_kw['height_ratios'][-2]
    ax_res_ylim = (-0.5*height_ratio*height, +0.5*height_ratio*height)
    ax_res[i+1].set(ylim=ax_res_ylim)
    #ax_res_height = np.abs(ax_res[i+1].get_ylim()[1] - ax_res[i+1].get_ylim()[0])

    ax_res[i+1].set(xlim=ax_spec[i+1].get_xlim())

ax_spec[0].set(ylim=(0,1.4e-14))
ax_res[0].set(xlim=ax_spec[0].get_xlim())
height = np.abs(np.diff(ax_spec[0].get_ylim()))

height_ratio = gridspec_kw['height_ratios'][1] / gridspec_kw['height_ratios'][0]
ax_res_ylim = (-0.5*height_ratio*height, +0.5*height_ratio*height)
ax_res[0].set(
    yticks=np.arange(-6e-16,2*6e-16,6e-16), 
    yticklabels=np.arange(-6e-16,2*6e-16,6e-16)/1e-14, 
    ylim=ax_res_ylim, 
    )
ax_res[-1].set(
    yticks=np.arange(-6e-16,2*6e-16,6e-16), 
    yticklabels=np.arange(-6e-16,2*6e-16,6e-16)/1e-15, 
    )

ax[-1].set(xlabel=r'Wavelength (nm)')
ax[1].set(ylabel=r'Residuals')
ax[0].set(ylabel=r'$F_\lambda\ (\mathrm{erg\ s^{-1}\ cm^{-2}\ nm^{-1}})$')
fig.savefig('./plots/bestfit_spec_zoom_ins_new.pdf')
plt.close(fig)

1.117321749249977
1.2569615071453117
1.4571432770130421
1.4987832601635789
1.3448857674678545
1.0863822730052382
0.9556506901825463
1.6700342566674881e-16


2.1604385688606084e-16

In [45]:
np.nanmean(d_spec_1.err[order,:])

1.942660350478406e-16